<a href="https://colab.research.google.com/github/hongducp850-maker/TTNT/blob/main/tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thuật toán Minimax ứng dụng vào bài toán tic tac toe
Minimax là thuật toán tìm chiến lược tối ưu trong trò chơi hai người chơi đối kháng (Zero-sum game).
Một người cố tối đa hóa điểm số (MAX), người còn lại cố tối thiểu hóa điểm số (MIN).

Tic Tac Toe là ví dụ điển hình:

AI đóng vai MAX (cố thắng → điểm cao)

Người chơi đóng vai MIN (cố làm AI thua → điểm thấp)
Mục tiêu của Minimax

Tìm nước đi tối ưu bằng cách:

Giả lập tất cả các khả năng có thể xảy ra

Duyệt toàn bộ cây trạng thái

Tính điểm mỗi trạng thái cuối (thắng, thua, hòa)

Lan truyền điểm ngược lại lên gốc để chọn nước tốt nhất
Cách hoạt động của Minimax
Cấu trúc của Minimax thường gồm hai hàm:
(1) maxValue(state)
AI (X) chọn nước đi tốt nhất cho MAX
v=max(v,minValue(result(state,action)))
(2) minValue(state)
Người chơi (O) cố giảm điểm MAX
v=min(v,maxValue(result(state,action)))
Điều kiện kết thúc
Nếu trạng thái là:

X thắng → utility = +1

O thắng → utility = -1

Hòa → utility = 0

Thuật toán dừng khi:

Có người thắng

Hoặc hết nước đi
Ứng dụng Minimax trong Tic Tac Toe

Đối với Tic Tac Toe:

Bảng 3×3: Minimax chạy được toàn bộ cây trạng thái, AI bất bại

Bảng 5×5, 10×10: Minimax thuần rất chậm, vì số trạng thái là:O((N2)!)
Ưu điểm của Minimax

Tìm chiến lược tối ưu 100%
Đảm bảo không thua (perfect play)
Đơn giản, dễ cài đặt

Nhược điểm

Chậm khi số trạng thái lớn (N > 3)
Tốn bộ nhớ
Bắt buộc duyệt toàn bộ cây trò chơi → không khả thi với NxN

In [ ]:
import copy
import math
import numpy as np

X = "X"
O = "O"
EMPTY = None
user = None
ai = None


# -------------------------------
# Tạo bảng NxN
# -------------------------------
def initial_state(N):
    return [[EMPTY for _ in range(N)] for _ in range(N)]


# -------------------------------
# Xác định người chơi tiếp theo
# -------------------------------
def player(board):
    count = sum(1 for row in board for cell in row if cell is not None)
    return ai if count % 2 != 0 else user


# -------------------------------
# Các bước đi khả dụng
# -------------------------------
def actions(board):
    res = set()
    N = len(board)
    for i in range(N):
        for j in range(N):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res


# -------------------------------
# Lấy bàn cờ sau khi đi một nước
# -------------------------------
def result(board, action):
    new_board = copy.deepcopy(board)
    (i, j) = action
    new_board[i][j] = player(board)
    return new_board


# -------------------------------
# Kiểm tra hàng thắng
# -------------------------------
def get_horizontal_winner(board):
    N = len(board)
    for row in board:
        if row[0] is not None and all(cell == row[0] for cell in row):
            return row[0]
    return None


# -------------------------------
# Kiểm tra cột thắng
# -------------------------------
def get_vertical_winner(board):
    N = len(board)
    for col in range(N):
        first = board[0][col]
        if first is not None and all(board[row][col] == first for row in range(N)):
            return first
    return None


# -------------------------------
# Kiểm tra đường chéo thắng
# -------------------------------
def get_diagonal_winner(board):
    N = len(board)

    # main diagonal
    first = board[0][0]
    if first is not None and all(board[i][i] == first for i in range(N)):
        return first

    # anti-diagonal
    first = board[0][N - 1]
    if first is not None and all(board[i][N - 1 - i] == first for i in range(N)):
        return first

    return None


# -------------------------------
# Xác định người thắng cuộc
# -------------------------------
def winner(board):
    return (
        get_horizontal_winner(board)
        or get_vertical_winner(board)
        or get_diagonal_winner(board)
        or None
    )


# -------------------------------
# Kiểm tra kết thúc trò chơi
# -------------------------------
def terminal(board):
    if winner(board) is not None:
        return True
    # còn ô trống → chưa kết thúc
    return not any(EMPTY in row for row in board)


# -------------------------------
# Giá trị của trạng thái cuối
# -------------------------------
def utility(board):
    w = winner(board)
    if w == X: return 1
    if w == O: return -1
    return 0


# -------------------------------
# Minimax: maxValue
# -------------------------------
def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v


# -------------------------------
# Minimax: minValue
# -------------------------------
def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v


# -------------------------------
# Minimax trả về nước đi tối ưu
# -------------------------------
def minimax(board):
    turn = player(board)

    best_move = None

    if turn == X:
        best_score = -math.inf
        for action in actions(board):
            score = minValue(result(board, action))
            if score > best_score:
                best_score = score
                best_move = action

    else:  # O
        best_score = math.inf
        for action in actions(board):
            score = maxValue(result(board, action))
            if score < best_score:
                best_score = score
                best_move = action

    return best_move


# -------------------------------
# GAME LOOP
# -------------------------------
if __name__ == "__main__":

    N = int(input("Nhập kích thước bảng N (ví dụ 3, 5, 10): "))
    board = initial_state(N)

    print("Chọn người chơi (X hoặc O):")
    user = input().strip()
    ai = O if user == X else X

    ai_turn = False

    while True:
        print(np.array(board))

        if terminal(board):
            w = winner(board)
            if w is None:
                print("Hòa!")
            else:
                print("Thắng:", w)
            break

        turn = player(board)

        # AI chơi
        if turn == ai:
            print("AI đang nghĩ...")
            move = minimax(board)
            board = result(board, move)

        # Người chơi
        else:
            while True:
                try:
                    print("Bạn đi (i, j):")
                    i = int(input("i = "))
                    j = int(input("j = "))

                    if 0 <= i < N and 0 <= j < N: # Validate input within board bounds
                        if board[i][j] == EMPTY:
                            board = result(board, (i, j))
                            break # Valid move, exit loop
                        else:
                            print("Ô đã có người đánh! Vui lòng chọn ô khác.")
                    else:
                        print(f"Tọa độ nằm ngoài bảng. Vui lòng nhập i, j trong khoảng từ 0 đến {N-1}.")
                except ValueError:
                    print("Đầu vào không hợp lệ. Vui lòng nhập số nguyên.")


Nhập kích thước bảng N (ví dụ 3, 5, 10): 3
Chọn người chơi (X hoặc O):
X
[[None None None]
 [None None None]
 [None None None]]
Bạn đi (i, j):
i = 3
j = 1
Tọa độ nằm ngoài bảng. Vui lòng nhập i, j trong khoảng từ 0 đến 2.
Bạn đi (i, j):
i = 0
j = 1
[[None 'X' None]
 [None None None]
 [None None None]]
AI đang nghĩ...
[[None 'X' None]
 [None None None]
 [None 'O' None]]
Bạn đi (i, j):
i = 2
j = 1
Ô đã có người đánh! Vui lòng chọn ô khác.
Bạn đi (i, j):
i = 1
j = 1
[[None 'X' None]
 [None 'X' None]
 [None 'O' None]]
AI đang nghĩ...
[['O' 'X' None]
 [None 'X' None]
 [None 'O' None]]
Bạn đi (i, j):
i = 2
j = 0
[['O' 'X' None]
 [None 'X' None]
 ['X' 'O' None]]
AI đang nghĩ...
[['O' 'X' 'O']
 [None 'X' None]
 ['X' 'O' None]]
Bạn đi (i, j):
i = 1 1
Đầu vào không hợp lệ. Vui lòng nhập số nguyên.
Bạn đi (i, j):
i = 1
j = 0
[['O' 'X' 'O']
 ['X' 'X' None]
 ['X' 'O' None]]
AI đang nghĩ...
[['O' 'X' 'O']
 ['X' 'X' 'O']
 ['X' 'O' None]]
Bạn đi (i, j):
i = 2
j = 2
[['O' 'X' 'O']
 ['X' 'X' 'O']
 ['X' 

Thuật toán cắt tỉa Alpha-beta với ứng dụng vào bài toán TicTacToe
Alpha-Beta là kỹ thuật tối ưu hóa Minimax.
Nó không thay đổi kết quả của Minimax → vẫn chính xác 100%.

Nó chỉ loại bỏ các nhánh không thể ảnh hưởng kết quả cuối, giúp tăng tốc cực mạnh.
Mục tiêu

Giảm số lượng trạng thái cần duyệt
→ giúp xử lý được các bảng lớn hơn như 5×5, 10×10.
Tham số Alpha và Beta

Alpha (α): giá trị tốt nhất mà MAX có thể đạt được

Beta (β): giá trị tốt nhất mà MIN có thể đạt được

Cắt tỉa xảy ra khi:α≥β
Ta có thể bỏ qua nhánh đó vì:

MAX biết MIN sẽ không để MAX đạt giá trị đó

MIN biết MAX sẽ né kết quả đó
Cách hoạt động
maxValue(state, α, β)
for action in actions:
    v = max(v, minValue(result, α, β))
    α = max(α, v)
    if α >= β:
        break   # cắt tỉa
minValue(state, α, β)
for action in actions:
    v = min(v, maxValue(result, α, β))
    β = min(β, v)
    if α >= β:
        break   # cắt tỉa
Ứng dụng Alpha-Beta trong Tic Tac Toe

Bảng 3×3: chạy cực nhanh, duyệt ~1,000 trạng thái thay vì ~250,000

Bảng 5×5 hoặc 10×10: có thể chạy được nếu kết hợp:

cắt tỉa alpha-beta

giới hạn độ sâu (depth limit)

heuristic đánh giá
Ưu điểm của Alpha-Beta

Nhanh hơn Minimax gấp 10–100 lần
Không thay đổi kết quả → vẫn là chiến lược tối ưu
Giảm số trạng thái phải xét
Có thể dùng cho bảng lớn hơn (5×5, 10×10)

Nhược điểm

Cài đặt phức tạp hơn Minimax thuần
Bảng NxN lớn vẫn quá nặng nếu không giới hạn độ sâu
Hiệu quả giảm nếu nước đi không sắp xếp trước

In [ ]:
import os
import math

# =============================
# GLOBAL CONFIG
# =============================
N = 5   # đổi thành 5, 10 hoặc bất kỳ N

# =============================
# HÀM KIỂM TRA NGƯỜI THẮNG
# =============================
def GetWinner(board):
    """
    Trả về 'X', 'O' nếu có người thắng, ngược lại trả về None.
    """
    # kiểm tra hàng
    for r in range(N):
        row = board[r*N:(r+1)*N]
        if len(set(row)) == 1:
            return row[0]

    # kiểm tra cột
    for c in range(N):
        col = [board[r*N + c] for r in range(N)]
        if len(set(col)) == 1:
            return col[0]

    # kiểm tra chéo chính
    diag1 = [board[i*N + i] for i in range(N)]
    if len(set(diag1)) == 1:
        return diag1[0]

    # kiểm tra chéo phụ
    diag2 = [board[i*N + (N-1-i)] for i in range(N)]
    if len(set(diag2)) == 1:
        return diag2[0]

    return None


# =============================
# LẤY CÁC Ô CÒN TRỐNG
# =============================
def GetAvailableCells(board):
    return [i for i in range(len(board)) if board[i] != "X" and board[i] != "O"]


# =============================
# THUẬT TOÁN MINIMAX + ALPHA-BETA
# =============================
def minimax(board, depth, alpha, beta, isMaximizing):
    winner = GetWinner(board)

    # Nếu có người thắng
    if winner == "X":
        return 10 - depth
    elif winner == "O":
        return -10 + depth

    # Hòa
    if len(GetAvailableCells(board)) == 0:
        return 0

    if isMaximizing:  # lượt X
        maxEval = -math.inf
        for cell in GetAvailableCells(board):
            board[cell] = "X"
            value = minimax(board, depth + 1, alpha, beta, False)
            board[cell] = cell

            maxEval = max(maxEval, value)
            alpha = max(alpha, value)

            if beta <= alpha:
                break  # cắt tỉa

        return maxEval

    else:  # lượt O
        minEval = +math.inf
        for cell in GetAvailableCells(board):
            board[cell] = "O"
            value = minimax(board, depth + 1, alpha, beta, True)
            board[cell] = cell

            minEval = min(minEval, value)
            beta = min(beta, value)

            if beta <= alpha:
                break  # cắt tỉa

        return minEval


# =============================
# TÌM NƯỚC ĐI TỐT NHẤT
# =============================
def FindBestMove(board, AI):
    bestVal = -math.inf if AI == "X" else +math.inf
    bestMove = None

    for cell in GetAvailableCells(board):
        board[cell] = AI
        value = minimax(board, 0, -math.inf, +math.inf, False if AI == "X" else True)
        board[cell] = cell

        if AI == "X" and value > bestVal:
            bestVal = value
            bestMove = cell
        elif AI == "O" and value < bestVal:
            bestVal = value
            bestMove = cell

    return bestMove


# =============================
# IN BÀN CỜ N×N
# =============================
def PrintBoard(board):
    os.system('cls' if os.name=='nt' else 'clear')
    for r in range(N):
        row = board[r*N:(r+1)*N]
        print(" | ".join(str(x) for x in row))
    print()


# =============================
# CHẠY TRÒ CHƠI
# =============================
def main():
    global N
    N = int(input("Nhập kích thước bàn cờ N×N: "))
    player = input("Bạn chọn X hay O? ").strip().upper()
    AI = "O" if player == "X" else "X"

    board = [i for i in range(N*N)]
    turn = "X"

    while True:
        PrintBoard(board)

        # AI đánh
        if turn == AI:
            print("AI đang suy nghĩ...")
            move = FindBestMove(board, AI)
            board[move] = AI
            turn = player

        else:  # Người chơi
            while True:
                try:
                    x = int(input("Chọn ô: "))
                    if x in board:
                        board[x] = player
                        turn = AI
                        break
                except:
                    pass
                PrintBoard(board)
                print("Ô không hợp lệ!")

        # kiểm tra thắng
        win = GetWinner(board)
        if win:
            PrintBoard(board)
            print(win, "THẮNG!")
            break

        # kiểm tra hòa
        if len(GetAvailableCells(board)) == 0:
            PrintBoard(board)
            print("HÒA!")
            break


if __name__ == "__main__":
    main()


Nhập kích thước bàn cờ N×N: 3
Bạn chọn X hay O? X
0 | 1 | 2
3 | 4 | 5
6 | 7 | 8

Chọn ô: 1
0 | X | 2
3 | 4 | 5
6 | 7 | 8

AI đang suy nghĩ...
O | X | 2
3 | 4 | 5
6 | 7 | 8

Chọn ô: 4
O | X | 2
3 | X | 5
6 | 7 | 8

AI đang suy nghĩ...
O | X | 2
3 | X | 5
6 | O | 8

Chọn ô: 3
O | X | 2
X | X | 5
6 | O | 8

AI đang suy nghĩ...
O | X | 2
X | X | O
6 | O | 8

Chọn ô: 2
O | X | X
X | X | O
6 | O | 8

AI đang suy nghĩ...
O | X | X
X | X | O
O | O | 8

Chọn ô: 8
O | X | X
X | X | O
O | O | X

HÒA!
